In [8]:
# import dependencies
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

In [9]:
# read in csv file & view first 5 rows
df = pd.read_csv("gdp_pcap.csv")
df.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,599,599,599,599,599,599,599,599,599,...,4800,4910,5030,5150,5270,5390,5520,5650,5780,5920
1,Angola,465,466,469,471,472,475,477,479,481,...,24.8k,25.3k,25.9k,26.4k,26.9k,27.4k,28k,28.5k,29.1k,29.6k
2,Albania,585,587,588,590,592,593,595,597,598,...,54k,54.6k,55.2k,55.8k,56.4k,56.9k,57.5k,58.1k,58.7k,59.2k
3,Andorra,1710,1710,1710,1720,1720,1720,1730,1730,1730,...,79.3k,79.5k,79.8k,80.1k,80.4k,80.7k,81k,81.2k,81.5k,81.8k
4,UAE,1420,1430,1430,1440,1450,1450,1460,1460,1470,...,92.5k,92.6k,92.6k,92.7k,92.8k,92.9k,92.9k,93k,93.1k,93.1k


In [10]:
stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] # load the CSS stylesheet

app = Dash(__name__, external_stylesheets=stylesheets) # initialize the app

In [17]:
# pivot table to organize by country, years, and gdpPerCapita
countryPerCap = df.melt(id_vars = 'country', 
                          value_vars = df.columns[1:],  #extracts first row of years
                          var_name = 'year',
                          value_name = 'gdpPercap')

# Convert 'gdpPercap' column to numeric
countryPerCap['gdpPercap'] = pd.to_numeric(countryPerCap['gdpPercap'], errors='coerce')

# Drop rows with NaN values in 'gdpPercap' column
countryPerCap = countryPerCap.dropna(subset=['gdpPercap'])

# Sort the pivot table by GDP per Capita
countryPerCap_sorted = countryPerCap.sort_values(by=['year', 'gdpPercap'])

# build a line chart using pivot table
fig_line_color = px.line(countryPerCap_sorted, 
                      x = 'year', 
                      y = 'gdpPercap',
                      color = 'country',
                      title = 'GDP per Capita by Country Through the Years')

In [29]:
app = Dash(__name__, external_stylesheets=stylesheets)  #initialize the app

years = df.columns[1:]   #set aside this list for the years

app.layout = html.Div([    
    html.H1("GDP Analysis Dashboard"),  #Header for title
    html.P("This data analyzes the GDP per Capita of different countries over time. To use the app, first select the countries you want to study. Then, select the range of years you want to look at. The final component shows a line chart of different countries GDP capita through the years."), #description under title
        dcc.Dropdown(      #use dcc to create dropdown menu
        id='country-dropdown',      #ID tag for dropdown
        options=[{'label': country, 'value': country} for country in df['country']],   #specifies to go through each country in the dataframe as the options for the dropdown
        multi=True,     #allows to click multiple countries
        placeholder="Select Countries",     #name of the dropdown before countries are selected
        className="six columns"          #formats dropdown to be on left half of screen
    ),
    

    dcc.RangeSlider(    #use dcc to create range slider 
        id='year-slider',  #ID tag for range slider
        marks={int(min(years)): str(int(min(years))), int(max(years)): str(int(max(years)))},   #shows markings on slider as minimum and maximum years from the dataframe
        min=int(min(years)),   #minimum as minimum from df years list
        max=int(max(years)),    #maximum as maximum from df years list
        step=1,   #steps up by 1
        value=[int(min(years)), int(max(years))],   
        className="six columns"   #formats range slider to be on right half of screen
    ),
    dcc.Graph(   #use dcc to greate graph that uses the plotly line chart from above
        id='gdp-line-chart',
        figure=fig_line_color,
        className="twelve columns"
    )
]

,className="row")

@app.callback(
    Output('gdp-line-chart', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('year-slider', 'value')]
)
def update_line_chart(selected_countries, selected_years):
    # Convert selected years to integers because they were strings
    selected_years = [int(year) for year in selected_years]
    
    # Filter DataFrame based on selected years
    filtered_df = countryPerCap_sorted[(countryPerCap_sorted['year'].astype(int) >= selected_years[0]) & (countryPerCap_sorted['year'].astype(int) <= selected_years[1])]
    
    # Filter DataFrame based on selected countries if any are selected
    if selected_countries:
        filtered_df = filtered_df[filtered_df['country'].isin(selected_countries)]
    
    # Create traces for each selected country
    traces = []
    if not filtered_df.empty:
        for country in filtered_df['country'].unique():
            country_data = filtered_df[filtered_df['country'] == country]
            trace = dict(
                x=country_data['year'],
                y=country_data['gdpPercap'],
                mode='lines',
                name=country
            )
            traces.append(trace)

    # Return updated figure
        return {
            'data': traces,
            'layout': dict(
                title='GDP per Capita by Country Through the Years',
                xaxis={'title': 'Year'},
                yaxis={'title': 'GDP per Capita'}
            )
        }


if __name__ == '__main__':
    app.run(jupyter_mode='tab', debug=True)

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>